This is the NLP segment and deals with only converting the news to sentiment values and saving them along the way. The Phase two of this the the stock prediction which is the second notebook in the folder which has the codde which merges the output of this with the stock data and trains the LSTM model.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import os
import pandas as pd
import numpy as np
np.random.seed(7)
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import Conv1D, GlobalMaxPooling1D,Embedding, Dropout,BatchNormalization
from xgboost import XGBClassifier
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
Using TensorFlow backend.


In [3]:
class data_manager:
  """
  This segment here imports the data. 
  It is designed to either import data using 
  pandas or numpy depending on the typr of file.

  


  """
  
  
  def fetch_data(self,path_to_data='',numpy=False):
    """
    This function fetches the data using pandas or numpy depending on the Value set
    and returns a data. It takes an additional string which is the location of the file to import the data.
    
    path_to_data : String 
    the directory of the file that needs to be fetched.

    numpy : Boolean
    When true it loads numpy data. Default is False to load the csv data using pandas.
    """
    if numpy:
      return np.load(path_to_data)
    else:
      return pd.read_csv(path_to_data,low_memory=False,parse_dates=[0])
  def saver(self,file_to_save,path_to_save=''):
    np.save(path_to_save,file_to_save)



importer=data_manager()
df=importer.fetch_data("drive/My Drive/Summer 2020/Finance/data/stocknews/Combined_News_DJIA.csv")
print(df.head(5))
#df = pd.read_csv("drive/My Drive/Summer 2020/Finance/data/stocknews/Combined_News_DJIA.csv",low_memory=False, parse_dates=[0])

full_stock = importer.fetch_data("drive/My Drive/Summer 2020/Finance/data/stocknews/upload_DJIA_table.csv")

#add the closing stock value to the df - this will be the y variable
df["Close"]=full_stock.Close

#show how the dataset looks like
#df.head(5)
print(df.shape)

        Date  ...                                              Top25
0 2008-08-08  ...           b"No Help for Mexico's Kidnapping Surge"
1 2008-08-11  ...  b"So this is what it's come to: trading sex fo...
2 2008-08-12  ...  b"BBC NEWS | Asia-Pacific | Extinction 'by man...
3 2008-08-13  ...  b'2006: Nobel laureate Aleksander Solzhenitsyn...
4 2008-08-14  ...  b'Philippines : Peace Advocate say Muslims nee...

[5 rows x 27 columns]
(1989, 28)


In [4]:
"""
This segment deals with cleaning the data. 
The web scrapped data leads to unwanted characters which 
causes unwanted characters which are cleaned in this segment.
We replace nan with empty string and we count the number of nans 
in the data to do a sanity check to make sure all nans are removed.
"""

nlp=df['Label']
nlp=nlp.to_numpy()  # we strip the labels and take it out if the equation before we clean the data.
df = df.drop(["Label"], axis=1)
print (df.isnull().sum())
df = df.replace(np.nan, ' ', regex=True)
scaler=MinMaxScaler(feature_range=(-1,1))

#sanity check
print(df.isnull().sum().sum())
df = df.replace('b\"|b\'|\\\\|\\\"', '', regex=True)
df.head(2)

Date     0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
Close    0
dtype: int64
0


,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,Close
0,2008-08-08,Georgia 'downs two Russian warplanes' as count...,BREAKING: Musharraf to be impeached.',Russia Today: Columns of troops roll into Sout...,Russian tanks are moving towards the capital o...,"Afghan children raped with 'impunity,' U.N. of...",150 Russian tanks have entered South Ossetia w...,"Breaking: Georgia invades South Ossetia, Russi...",The 'enemy combatent' trials are nothing but a...,Georgian troops retreat from S. Osettain capit...,Did the U.S. Prep Georgia for War with Russia?',Rice Gives Green Light for Israel to Attack Ir...,Announcing:Class Action Lawsuit on Behalf of A...,So---Russia and Georgia are at war and the NYT...,China tells Bush to stay out of other countrie...,Did World War III start today?',Georgia Invades South Ossetia - if Russia gets...,Al-Qaeda Faces Islamist Backlash',Condoleezza Rice: The US would not act to prev...,This is a busy day: The European Union has ap...,"Georgia will withdraw 1,000 soldiers from Iraq...",Why the Pentagon Thinks Attacking Iran is a Ba...,Caucasus in crisis: Georgia invades South Osse...,Indian shoe manufactory - And again in a seri...,Visitors Suffering from Mental Illnesses Banne...,No Help for Mexico's Kidnapping Surge,11734.32031
1,2008-08-11,Why wont America and Nato help us? If they won...,Bush puts foot down on Georgian conflict',Jewish Georgian minister: Thanks to Israeli tr...,Georgian army flees in disarray as Russians ad...,Olympic opening ceremony fireworks 'faked',What were the Mossad with fraudulent New Zeala...,Russia angered by Israeli military sale to Geo...,An American citizen living in S.Ossetia blames...,Welcome To World War IV! Now In High Definition!',"Georgia's move, a mistake of monumental propor...",Russia presses deeper into Georgia; U.S. says ...,Abhinav Bindra wins first ever Individual Olym...,U.S. ship heads for Arctic to define territory',Drivers in a Jerusalem taxi station threaten t...,The French Team is Stunned by Phelps and the 4...,Israel and the US behind the Georgian aggressi...,"Do not believe TV, neither Russian nor Georgia...",Riots are still going on in Montreal (Canada) ...,China to overtake US as largest manufacturer',War in South Ossetia [PICS]',Israeli Physicians Group Condemns State Torture',Russia has just beaten the United States over...,Perhaps *the* question about the Georgia - Rus...,Russia is so much better at war',So this is what it's come to: trading sex for ...,11782.34961


In [ ]:

class sentiment:
  """
  This segment deals with getting the NLP sentiment values for each headline. 
  We traverse through earch day and each of the 25 headlines for the day and 
  calculte the sentiment value. then we generate a tabular data with just 
  the compund values for each news for each day. we then save the calulated 
  values for being used for MLP prediction.

  The function class provides us with a numpy data array with the compound values for each news chunk.
  

  news_data : pandas dataframe
  """
  def __init__(self,news_data):
    self.data=news_data
  def sentiment_analysis(self):
    results=[]
    for y in range(1,26):
        res=[]
        print(y)
        for x in self.data['Top'+str(y)]:
            
            pol_score = SIA().polarity_scores(x) # run analysis
            
            res.append(pol_score['compound'])
            
        results.append(res)

    print(results)
    sentiment_values=np.array(results)  
    return sentiment_values
sentiment_analyzer=sentiment(df)
NLP_values=sentiment_analyzer.sentiment_analysis()
#importer.saver("C:/Users/byakuya/Downloads/stocknews/nlp_compound.npy",NLP_inputs)



'''results=[]
for y in range(1,26):
    res=[]
    print(y)
    for x in df['Top'+str(y)]:
        
        pol_score = SIA().polarity_scores(x) # run analysis
        
        res.append(pol_score['compound'])
        
    results.append(res)

print(results)
NLP_inputs=np.array(results)  
np.save("C:/Users/byakuya/Downloads/stocknews/nlp_compound.npy",NLP_inputs)'''
#np.save('C:/Users/byakuya/Downloads/stocknews/nlp_values.npy',nlp) # this was to check the output of the actual labels.

1


KeyboardInterrupt: ignored

In [5]:
"""
First we import the compound values. 
The path is the current path for the NLP
sentiment values for each day stored in the drive.
We then scale and split the data to test and train. the current split is 60% train and 40% Test.
"""

dat=importer.fetch_data("drive/My Drive/Summer 2020/Finance/data/stocknews/nlp_compound.npy",numpy=True).T
scaler.fit(dat)
X_train, X_test, y_train, y_test = train_test_split( dat, nlp, test_size=0.4)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


"""
Just for comparison I also used random forests 
to predict the values and print its final score.
"""
clf = RandomForestClassifier(max_depth=12 )
clf.fit(X_train,y_train)
print (clf.score(X_test,y_test))

0.5251256281407035


In [ ]:
"""
Similarly, I also used Extreme Gradient boosting (XGBoost) for comapring with my model.
"""
param_dist = {'objective':'binary:logistic', 'n_estimators':3000,'max_depth':20,'learning_rate':0.01}

clf = XGBClassifier(**param_dist)

clf.fit(X_train, y_train,
        
        verbose=True)

#evals_result = clf.evals_result()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
"""
A part of XGBoost.
"""
X_test.shape
print(np.sum(y_test),len(y_test))
out=clf.predict(X_test)
out1=clf.predict(X_train)
print(clf.score(X_test,y_test))
#np.save('drive/My Drive/downloads/stocknews/out_test.npy',out)
#np.save('drive/My Drive/downloads/stocknews/out_train.npy',out1)
print(out )

424 796
0.4949748743718593
[1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 0 1
 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0
 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 1 0
 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 1
 1 0 1 1 0 0 0 0 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1
 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0
 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0
 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1
 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1
 0 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 0 1 1 1 0 

In [20]:
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf
from keras.layers.advanced_activations import LeakyReLU
from tensorflow import keras
loss_fn = keras.losses.BinaryCrossentropy(label_smoothing=0.01)
model = Sequential()
tf.random.set_seed(0)
#initializer = tf.keras.initializers.GlorotUniform()
#model.add(Dropout(0.2))
model.add(Dense(16, input_dim=25))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(32))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64))
model.add(LeakyReLU(0.1))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(Dense(32))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(16))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(8))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(5))
model.add(LeakyReLU(0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=loss_fn, optimizer=Adam(learning_rate=0.0005),metrics=["accuracy"])
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
print(model.summary())
model.fit(X_train,y_train,epochs=100,validation_split=0.2,batch_size=128,callbacks=callbacks_list,shuffle=False,)
print(model.evaluate(X_test,y_test,batch_size=128))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 16)                416       
_________________________________________________________________
leaky_re_lu_43 (LeakyReLU)   (None, 16)                0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 32)                544       
_________________________________________________________________
leaky_re_lu_44 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 32)               

In [22]:
model.load_weights("weights-improvement-05-0.57.hdf5")
print(model.evaluate(X_test,y_test,batch_size=128))

796/796 [==============================] - 0s 14us/step
[0.6915075704080975, 0.536432147026062]


In [ ]:
"""
This is where I have built my model. This a MLP
model which saves the best weights WRT to Validation 
accuracy. The model includes blocks of Dense, Batch, 
Dropout layers initially increasing in number of nodes 
and then converging to single node to predict 0,1.
The model uses 20% validation split. 
"""
def create_model(units,drop):
  model = Sequential()

  #model.add(Dropout(0.2))
  model.add(Dense(units, activation='relu', input_dim=25))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())
  model.add(Dense(units*2, activation='relu'))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  """
  model.add(Dense(units*4,activation='relu'))
  #model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Dense(units*8, activation='relu'))
  #model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Dense(units*4,activation='relu'))
  #model.add(Dropout(drop))
  model.add(BatchNormalization())
  """
  model.add(Dense(units*2, activation='relu'))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Dense(units, activation='relu'))
  model.add(Dropout(drop))
  model.add(BatchNormalization())
  model.add(Dense(100,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  
  model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
  print(model.summary())
  return model

model=create_model(256,drop=0.2)
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
  
model.fit(X_train,y_train,epochs=100,validation_split=0.2,batch_size=12,callbacks=callbacks_list)
print(model.evaluate(X_test,y_test,batch_size=128))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               6656      
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
dense_8 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 512)               2048      
_________________________________________________________________
dense_9 (Dense)              (None, 512)              

In [ ]:
"""
This is a dummy model with same architecture to load pretrained 
weights so that we can use the best trained weight set. One of the 
best models is included in the drive in the weights folder. Which produces 74% accuracy.
"""


model=create_model(256,0.2)
model.load_weights("weights-improvement-05-0.58.hdf5")
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 256)               6656      
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 256)               1024      
_________________________________________________________________
dense_14 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 512)               2048      
_________________________________________________________________
dense_15 (Dense)             (None, 512)              

In [ ]:
"""
This part predicts and calculates the accuracy 
and saves the value for the predictions for the 
entire dataset for modifying the stock data to improve training.
"""

print(model.evaluate(X_test,y_test,batch_size=128))
#out=model.predict(dat)
#out1=model.predict(X_train)
#print(clf.score(X_test,y_test))
#np.save('drive/My Drive/Summer 2020/Finance/data/stocknews/out.npy',out)
#print(out)
#np.save('drive/My Drive/downloads/stocknews/out_train.npy',out1)


796/796 [==============================] - 0s 147us/step
[0.7304728255799068, 0.5100502371788025]
